In [2]:
import pandas as pd
import numpy as np

import xgboost as xgb
from sklearn.model_selection import GridSearchCV

import seaborn as sns

In [2]:
r2 = []
rmse = []

In [32]:
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
encoder = ce.OrdinalEncoder(cols=["ID_FINCA", "ID_ZONA", "ID_ESTACION", "ALTITUD", "VARIEDAD", "COLOR", "TIPO", "MODO", "SUPERFICIE"])


In [33]:
df = pd.read_csv("df_train_tractat.csv").drop(columns=['ALTITUD_MIN', 'ALTITUD_DIF'])
df = encoder.fit_transform(df)

In [34]:
df2 = pd.read_csv("./UH_2023/UH_2023_TRAIN.txt", sep = "|")
df2 = encoder.fit_transform(df2)

In [35]:
X_train = df[df['CAMPAÑA']!='22'].drop(columns=['PRODUCCION'])
y_train = df[df['CAMPAÑA']!='22']['PRODUCCION']
X_train
y_train

0       22215.0
1       22215.0
2       20978.0
3       40722.0
4       14126.0
         ...   
8521    28160.1
8522    41310.0
8523    45420.0
8524    56140.0
8525    13869.9
Name: PRODUCCION, Length: 8526, dtype: float64

In [37]:
X_test = df2[df2['CAMPAÑA']==22].drop(columns=['PRODUCCION'])
X_test

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE
8526,22,913,105,19,77,2,1,1,1,2
8527,22,914,105,19,77,2,1,1,1,3
8528,22,12,7,2,4,5,2,1,1,4
8529,22,13,7,2,4,2,1,1,1,1011
8530,22,13,7,2,4,6,1,1,1,1012
...,...,...,...,...,...,...,...,...,...,...
9596,22,909,104,18,7,8,1,1,1,869
9597,22,911,104,18,7,2,1,1,1,870
9598,22,911,104,18,7,5,1,1,1,810
9599,22,912,104,18,7,6,1,1,1,1211


In [39]:
gsc_f = xgb.XGBRegressor(n_estimators =300,
                        learning_rate= 0.15,
                        max_depth= 20,
                        min_child_weight = 1,
                        gamma = 0.0,
                        colsample_bytree=0.4)

grid_result_f = gsc_f.fit(X_train, y_train)

In [61]:
df_aux = pd.read_csv("./UH_2023/UH_2023_TRAIN.txt", sep = "|")
df_aux= df_aux[df_aux['CAMPAÑA']==22]
df_aux['PRODUCCION'] = grid_result_f.predict(X_test)
df_aux[df_aux['PRODUCCION'] < 0]['PRODUCCION'] = 0
mask = (df['PRODUCCION']<0)*1
df_aux.loc[mask, 'PRODUCCION'] = 0
df_aux = df_aux.sort_values(by=["ID_FINCA", "VARIEDAD", "MODO", "TIPO", "COLOR", "SUPERFICIE"])
df_aux = df_aux.reindex(columns=["ID_FINCA", "VARIEDAD", "MODO", "TIPO", "COLOR", "SUPERFICIE", "PRODUCCION"])
df_aux.head(50)

<ipython-input-61-bfed3a86148c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aux[df_aux['PRODUCCION'] < 0]['PRODUCCION'] = 0


KeyError: "None of [Int64Index([0, 0, 0, 0, 0, 0, 0, 0, 0, 0,\n            ...\n            0, 0, 0, 0, 0, 0, 0, 0, 0, 0],\n           dtype='int64', length=8526)] are in the [index]"

In [48]:
df_aux["PRODUCCION"] = df_aux["PRODUCCION"].apply(lambda x: '{:.2f}'.format(x) if type(x) is int or type(x) is float else x)
df_aux["SUPERFICIE"] = df_aux["SUPERFICIE"].apply(lambda x: '{:.2f}'.format(x) if type(x) is int or type(x) is float else x)

def fun(x): 
    x=str(x)
    if len(x) == 0:
        return '"' + '000' +'"'
    elif len(x) == 1:
        return '"' + '00' + x +'"'
    elif len(x) == 2:
        return '"' + '0' + x +'"'
    elif len(x) == 3:
        return '"' + x +'"'

def fun2(x): 
    x='"' + str(x) +'"'
    return x

df_aux["VARIEDAD"] = df_aux["VARIEDAD"].apply(fun)
df_aux["TIPO"] = df_aux["TIPO"].apply(fun2)


name_file = "UH2023_Universitat Autònoma de Barcelona (UAB)_Farts del vi_1.txt"
df_aux.to_csv(name_file, sep = "|", header=False, index=False)

**TRAIN I TEST ALEATORI (SENSE ALTITUD, CAMPANYA NI PRODUCCIÓ CLAR)**

In [31]:
df = pd.read_csv("/content/df_train_tractat.csv")
#df.drop(columns=[df.columns[0]], inplace=True)
#df[df['SUPERFICIE']<0]

In [4]:
X = df.drop(columns = ['PRODUCCION', 'ALTITUD'])
y = df['PRODUCCION']

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X, y,
                                                   test_size= 0.15,
                                                   shuffle= True, #shuffle the data to avoid bias
                                                   random_state= 0)

In [6]:
gsc1 = GridSearchCV(
            estimator=xgb.XGBRegressor(),
            param_grid={"n_estimators": [ 500],
                        "learning_rate": [0.15],
                        "max_depth": [20],
                        "min_child_weight": [1],
                        "gamma":[ 0.0],
                        "colsample_bytree":[0.4],},
            cv=5, scoring='r2', verbose=0, n_jobs=-1)

grid_result1 = gsc1.fit(X_train, y_train)
#grid_result1_1 = gsc1.fit(X,y)


In [7]:
print( grid_result1.score(X_train,y_train), grid_result1.score(X_test, y_test))
results = (grid_result1.score(X_train,y_train), grid_result1.score(X_test, y_test))
r2.append(results)

0.9999992561341488 0.8178248625881704


In [8]:
gsc1_1 = GridSearchCV(
            estimator=xgb.XGBRegressor(),
            param_grid={"n_estimators": [ 500],
                        "learning_rate": [0.15],
                        "max_depth": [20],
                        "min_child_weight": [1],
                        "gamma":[ 0.0],
                        "colsample_bytree":[0.4],},
            cv=5, scoring='neg_root_mean_squared_error', verbose=0, n_jobs=-1)

grid_result1_1 = gsc1_1.fit(X_train, y_train)
#grid_result1_1 = gsc1.fit(X,y)

In [9]:
print( grid_result1_1.score(X_train,y_train), grid_result1_1.score(X_test, y_test))
results = (grid_result1_1.score(X_train,y_train), grid_result1_1.score(X_test, y_test))
rmse.append(results)

-11.42461684042509 -5715.316533858098


In [10]:
#grid_result1.best_params_

**TRAIN ANYS ABANS 2021 I TEST 2021 (SENSE ALTITUD, NI PRODUCCIÓ CLAR)**

In [11]:
X = df.drop(columns = ['PRODUCCION', 'ALTITUD'])
X_train = X[X['CAMPAÑA'] != 21]
y_train = df[df['CAMPAÑA'] != 21]['PRODUCCION']

In [12]:
X_test = X[X['CAMPAÑA'] == 21]
y_test = df[df['CAMPAÑA'] == 21]['PRODUCCION']

In [13]:
gsc2 = GridSearchCV(
            estimator=xgb.XGBRegressor(),
            param_grid={"n_estimators": [300],
                        "learning_rate": [0.15],
                        "max_depth": [20],
                        "min_child_weight": [1],
                        "gamma":[ 0.0],
                        "colsample_bytree":[0.4],},
            cv=5, scoring='r2', verbose=0, n_jobs=-1)

grid_result2 = gsc2.fit(X_train, y_train)

In [14]:
print( grid_result2.score(X_train,y_train), grid_result2.score(X_test, y_test))
results = (grid_result2.score(X_train,y_train), grid_result2.score(X_test, y_test))
r2.append(results)

0.9999985998053591 0.785189512327019


In [15]:
gsc2_1 = GridSearchCV(
            estimator=xgb.XGBRegressor(),
            param_grid={"n_estimators": [300],
                        "learning_rate": [0.15],
                        "max_depth": [20],
                        "min_child_weight": [1],
                        "gamma":[ 0.0],
                        "colsample_bytree":[0.4],},
            cv=5, scoring='neg_root_mean_squared_error', verbose=0, n_jobs=-1)

grid_result2_1 = gsc2_1.fit(X_train, y_train)

In [16]:
print( grid_result2_1.score(X_train,y_train), grid_result2_1.score(X_test, y_test))
results = (grid_result2_1.score(X_train,y_train), grid_result2_1.score(X_test, y_test))
rmse.append(results)

-15.633273106167826 -6322.651292719792


In [17]:
#grid_result2.best_params_

**TRAIN I TEST ALEATORI (SENSE ALTITUD, CAMPAÑA, NI PRODUCCIÓ CLAR)**

In [18]:
X = df.drop(columns = ['PRODUCCION', 'ALTITUD', 'CAMPAÑA'])
y = df['PRODUCCION']

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X, y,
                                                   test_size= 0.15,
                                                   shuffle= True, #shuffle the data to avoid bias
                                                   random_state= 0)

In [20]:
gsc3 = GridSearchCV(
            estimator=xgb.XGBRegressor(),
            param_grid={"n_estimators": [100],
                        "learning_rate": [0.15],
                        "max_depth": [100],
                        "min_child_weight": [1],
                        "gamma":[ 0.0],
                        "colsample_bytree":[0.4],},
            cv=5, scoring='r2', verbose=0, n_jobs=-1)

grid_result3 = gsc3.fit(X_train, y_train)

In [21]:
print( grid_result3.score(X_train,y_train), grid_result3.score(X_test, y_test))
results = (grid_result3.score(X_train,y_train), grid_result3.score(X_test, y_test))
r2.append(results)

0.9903187624290571 0.8253618181314967


In [22]:
gsc3_1 = GridSearchCV(
            estimator=xgb.XGBRegressor(),
            param_grid={"n_estimators": [100],
                        "learning_rate": [0.15],
                        "max_depth": [100],
                        "min_child_weight": [1],
                        "gamma":[ 0.0],
                        "colsample_bytree":[0.4],},
            cv=5, scoring='neg_root_mean_squared_error', verbose=0, n_jobs=-1)

grid_result3_1 = gsc3_1.fit(X_train, y_train)

In [23]:
print( grid_result3_1.score(X_train,y_train), grid_result3_1.score(X_test, y_test))
results = (grid_result3_1.score(X_train,y_train), grid_result3_1.score(X_test, y_test))
rmse.append(results)

-1303.3461439260245 -5595.840612606942


In [24]:
#grid_result3.best_params_

In [25]:
#grid_result3_1.best_params_

**TRAIN I TEST ALEATORI (SENSE ALTITUD, SUPERFICIE, NI PRODUCCIÓ CLAR)**

In [26]:
X = df.drop(columns = ['PRODUCCION', 'ALTITUD', 'SUPERFICIE'])
y = df['PRODUCCION']

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X, y,
                                                   test_size= 0.15,
                                                   shuffle= True, #shuffle the data to avoid bias
                                                   random_state= 0)

In [28]:
gsc4 = GridSearchCV(
            estimator=xgb.XGBRegressor(),
            param_grid={"n_estimators": [100],
                        "learning_rate": [0.15],
                        "max_depth": [20],
                        "min_child_weight": [1],
                        "gamma":[ 0.0],
                        "colsample_bytree":[0.4],},
            cv=5, scoring='r2', verbose=0, n_jobs=-1)

grid_result4 = gsc4.fit(X_train, y_train)

In [29]:
print( grid_result4.score(X_train,y_train), grid_result4.score(X_test, y_test))
results = (grid_result4.score(X_train,y_train), grid_result4.score(X_test, y_test))
r2.append(results)

0.9743104800597309 0.7657780012920226


In [30]:
gsc4_1 = GridSearchCV(
            estimator=xgb.XGBRegressor(),
            param_grid={"n_estimators": [100],
                        "learning_rate": [0.15],
                        "max_depth": [20],
                        "min_child_weight": [1],
                        "gamma":[ 0.0],
                        "colsample_bytree":[0.4],},
            cv=5, scoring='neg_root_mean_squared_error', verbose=0, n_jobs=-1)

grid_result4_1 = gsc4_1.fit(X_train, y_train)

In [31]:
print( grid_result4_1.score(X_train,y_train), grid_result4_1.score(X_test, y_test))
results = (grid_result4_1.score(X_train,y_train), grid_result4_1.score(X_test, y_test))
rmse.append(results)

-2123.109174125441 -6480.515841173995


In [32]:
#grid_result4.best_params_

**TRAIN PRE 2021 I TEST 2021 (SENSE ALTITUD, CAMPAÑA, NI PRODUCCIÓ CLAR)**

In [33]:
X_train = df[df['CAMPAÑA'] != 21].drop(columns = ['PRODUCCION', 'ALTITUD', 'CAMPAÑA'])
y_train = df[df['CAMPAÑA'] != 21]['PRODUCCION']

In [34]:
X_test = df[df['CAMPAÑA'] == 21].drop(columns = ['PRODUCCION', 'ALTITUD', 'CAMPAÑA'])
y_test = df[df['CAMPAÑA'] == 21]['PRODUCCION']

In [35]:
gsc5 = GridSearchCV(
            estimator=xgb.XGBRegressor(),
            param_grid={"n_estimators": [ 300],
                        "learning_rate": [0.15],
                        "max_depth": [40],
                        "min_child_weight": [1],
                        "gamma":[ 0.0],
                        "colsample_bytree":[0.4],},
            cv=5, scoring='r2', verbose=0, n_jobs=-1)

grid_result5 = gsc5.fit(X_train, y_train)

In [36]:
print( grid_result5.score(X_train,y_train), grid_result5.score(X_test, y_test))
results = (grid_result5.score(X_train,y_train), grid_result5.score(X_test, y_test))
r2.append(results)

0.9992726156128926 0.7769533794143471


In [37]:
gsc5_1 = GridSearchCV(
            estimator=xgb.XGBRegressor(),
            param_grid={"n_estimators": [ 300],
                        "learning_rate": [0.15],
                        "max_depth": [40],
                        "min_child_weight": [1],
                        "gamma":[ 0.0],
                        "colsample_bytree":[0.4],},
            cv=5, scoring='neg_root_mean_squared_error', verbose=0, n_jobs=-1)

grid_result5_1 = gsc5_1.fit(X_train, y_train)

In [38]:
print( grid_result5_1.score(X_train,y_train), grid_result5_1.score(X_test, y_test))
results = (grid_result5_1.score(X_train,y_train), grid_result5_1.score(X_test, y_test))
rmse.append(results)

-356.31794649555417 -6442.720832617403


In [39]:
#grid_result5.best_params_

**TRAIN PRE 2021 I TEST 2021 (SENSE ALTITUD, ESTACIÓ, NI PRODUCCIÓ CLAR)**

In [40]:
X_train = df[df['CAMPAÑA'] != 21].drop(columns = ['PRODUCCION', 'ALTITUD', 'ID_ESTACION'])
y_train = df[df['CAMPAÑA'] != 21]['PRODUCCION']

In [41]:
X_test = df[df['CAMPAÑA'] == 21].drop(columns = ['PRODUCCION', 'ALTITUD', 'ID_ESTACION'])
y_test = df[df['CAMPAÑA'] == 21]['PRODUCCION']

In [42]:
gsc6 = GridSearchCV(
            estimator=xgb.XGBRegressor(),
            param_grid={"n_estimators": [300],
                        "learning_rate": [0.15],
                        "max_depth": [40],
                        "min_child_weight": [1],
                        "gamma":[ 0.0],
                        "colsample_bytree":[0.4],},
            cv=5, scoring='r2', verbose=0, n_jobs=-1)

grid_result6 = gsc6.fit(X_train, y_train)

In [43]:
print( grid_result6.score(X_train,y_train), grid_result6.score(X_test, y_test))
results = (grid_result6.score(X_train,y_train), grid_result6.score(X_test, y_test))
r2.append(results)

0.999999699878823 0.7717865752773425


In [44]:
gsc6_1 = GridSearchCV(
            estimator=xgb.XGBRegressor(),
            param_grid={"n_estimators": [300],
                        "learning_rate": [0.15],
                        "max_depth": [40],
                        "min_child_weight": [1],
                        "gamma":[ 0.0],
                        "colsample_bytree":[0.4],},
            cv=5, scoring='neg_root_mean_squared_error', verbose=0, n_jobs=-1)

grid_result6_1 = gsc6_1.fit(X_train, y_train)

In [45]:
print( grid_result6_1.score(X_train,y_train), grid_result6_1.score(X_test, y_test))
results = (grid_result6_1.score(X_train,y_train), grid_result6_1.score(X_test, y_test))
rmse.append(results)

-7.237757529993546 -6516.915410157708


In [46]:
#grid_result6.best_params_

**TRAIN PRE 2021 I TEST 2021 (SENSE ALTITUD, SUPERFICIE, NI PRODUCCIÓ CLAR)**

In [47]:
X_train = df[df['CAMPAÑA'] != 21].drop(columns = ['PRODUCCION', 'ALTITUD', 'SUPERFICIE'])
y_train = df[df['CAMPAÑA'] != 21]['PRODUCCION']

In [48]:
X_test = df[df['CAMPAÑA'] == 21].drop(columns = ['PRODUCCION', 'ALTITUD', 'SUPERFICIE'])
y_test = df[df['CAMPAÑA'] == 21]['PRODUCCION']

In [49]:
gsc7 = GridSearchCV(
            estimator=xgb.XGBRegressor(),
            param_grid={"n_estimators": [100],
                        "learning_rate": [0.15],
                        "max_depth": [60],
                        "min_child_weight": [1],
                        "gamma":[ 0.0],
                        "colsample_bytree":[0.4],},
            cv=5, scoring='r2', verbose=0, n_jobs=-1)

grid_result7 = gsc7.fit(X_train, y_train)

In [50]:
print( grid_result7.score(X_train,y_train), grid_result7.score(X_test, y_test))
results = (grid_result7.score(X_train,y_train), grid_result7.score(X_test, y_test))
r2.append(results)

0.9721059456678479 0.7795763774538554


In [51]:
gsc7_1 = GridSearchCV(
            estimator=xgb.XGBRegressor(),
            param_grid={"n_estimators": [100],
                        "learning_rate": [0.15],
                        "max_depth": [60],
                        "min_child_weight": [1],
                        "gamma":[ 0.0],
                        "colsample_bytree":[0.4],},
            cv=5, scoring='neg_root_mean_squared_error', verbose=0, n_jobs=-1)

grid_result7_1 = gsc7_1.fit(X_train, y_train)

In [52]:
print( grid_result7_1.score(X_train,y_train), grid_result7_1.score(X_test, y_test))
results = (grid_result7_1.score(X_train,y_train), grid_result7_1.score(X_test, y_test))
rmse.append(results)

-2206.5386072009433 -6404.726036792351


In [53]:
#grid_result7.best_params_

**TRAIN 14-19+21 I TEST 2020 (SENSE ALTITUD, SUPERFICIE, NI PRODUCCIÓ CLAR)**

In [54]:
X_train = df[df['CAMPAÑA'] != 20].drop(columns = ['PRODUCCION', 'ALTITUD', 'SUPERFICIE'])
y_train = df[df['CAMPAÑA'] != 20]['PRODUCCION']

In [55]:
X_test = df[df['CAMPAÑA'] == 20].drop(columns = ['PRODUCCION', 'ALTITUD', 'SUPERFICIE'])
y_test = df[df['CAMPAÑA'] == 20]['PRODUCCION']

In [56]:
gsc8 = GridSearchCV(
            estimator=xgb.XGBRegressor(),
            param_grid={"n_estimators": [100],
                        "learning_rate": [0.15],
                        "max_depth": [20],
                        "min_child_weight": [1],
                        "gamma":[ 0.0],
                        "colsample_bytree":[0.4],},
            cv=5, scoring='r2', verbose=0, n_jobs=-1)

grid_result8 = gsc8.fit(X_train, y_train)

In [57]:
print( grid_result8.score(X_train,y_train), grid_result8.score(X_test, y_test))
results = (grid_result8.score(X_train,y_train), grid_result8.score(X_test, y_test))
r2.append(results)

0.9726474615267175 0.7752420512855773


In [58]:
gsc8_1 = GridSearchCV(
            estimator=xgb.XGBRegressor(),
            param_grid={"n_estimators": [100],
                        "learning_rate": [0.15],
                        "max_depth": [20],
                        "min_child_weight": [1],
                        "gamma":[ 0.0],
                        "colsample_bytree":[0.4],},
            cv=5, scoring='neg_root_mean_squared_error', verbose=0, n_jobs=-1)

grid_result8_1 = gsc8_1.fit(X_train, y_train)

In [59]:
print( grid_result8_1.score(X_train,y_train), grid_result8_1.score(X_test, y_test))
results = (grid_result8_1.score(X_train,y_train), grid_result8_1.score(X_test, y_test))
rmse.append(results)

-2190.7390166606096 -6364.039637393608


In [60]:
#grid_result8.best_params_

**TRAIN 14-19+21 I TEST 2020 (SENSE ALTITUD, CAMPAÑA, NI PRODUCCIÓ CLAR)**

In [61]:
X_train = df[df['CAMPAÑA'] != 20].drop(columns = ['PRODUCCION', 'ALTITUD', 'CAMPAÑA'])
y_train = df[df['CAMPAÑA'] != 20]['PRODUCCION']

In [62]:
X_test = df[df['CAMPAÑA'] == 20].drop(columns = ['PRODUCCION', 'ALTITUD', 'CAMPAÑA'])
y_test = df[df['CAMPAÑA'] == 20]['PRODUCCION']

In [63]:
gsc9 = GridSearchCV(
            estimator=xgb.XGBRegressor(),
            param_grid={"n_estimators": [300],
                        "learning_rate": [0.15],
                        "max_depth": [20],
                        "min_child_weight": [1],
                        "gamma":[ 0.0],
                        "colsample_bytree":[0.4],},
            cv=5, scoring='r2', verbose=0, n_jobs=-1)

grid_result9 = gsc9.fit(X_train, y_train)

In [64]:
print( grid_result9.score(X_train,y_train), grid_result9.score(X_test, y_test))
results = (grid_result9.score(X_train,y_train), grid_result9.score(X_test, y_test))
r2.append(results)

0.9992600655510594 0.7535613992274898


In [65]:
gsc9_1 = GridSearchCV(
            estimator=xgb.XGBRegressor(),
            param_grid={"n_estimators": [300],
                        "learning_rate": [0.15],
                        "max_depth": [20],
                        "min_child_weight": [1],
                        "gamma":[ 0.0],
                        "colsample_bytree":[0.4],},
            cv=5, scoring='neg_root_mean_squared_error', verbose=0, n_jobs=-1)

grid_result9_1 = gsc9_1.fit(X_train, y_train)

In [66]:
print( grid_result9_1.score(X_train,y_train), grid_result9_1.score(X_test, y_test))
results = (grid_result9_1.score(X_train,y_train), grid_result9_1.score(X_test, y_test))
rmse.append(results)

-360.32006551523733 -6663.9190550972535


In [67]:
#grid_result9.best_params_

**TRAIN 14-20 I TEST 2021 (SENSE ALTITUD, ZONA, NI PRODUCCIÓ CLAR)**

In [68]:
X_train = df[df['CAMPAÑA'] != 21].drop(columns = ['PRODUCCION', 'ALTITUD', 'ID_ZONA'])
y_train = df[df['CAMPAÑA'] != 21]['PRODUCCION']

In [69]:
X_test = df[df['CAMPAÑA'] == 21].drop(columns = ['PRODUCCION', 'ALTITUD', 'ID_ZONA'])
y_test = df[df['CAMPAÑA'] == 21]['PRODUCCION']

In [70]:
gsc10 = GridSearchCV(
            estimator=xgb.XGBRegressor(),
            param_grid={"n_estimators": [300],
                        "learning_rate": [0.15],
                        "max_depth": [40],
                        "min_child_weight": [1],
                        "gamma":[ 0.0],
                        "colsample_bytree":[0.4],},
            cv=5, scoring='r2', verbose=0, n_jobs=-1)

grid_result10 = gsc10.fit(X_train, y_train)

In [71]:
print( grid_result10.score(X_train,y_train), grid_result10.score(X_test, y_test))
results = (grid_result10.score(X_train,y_train), grid_result10.score(X_test, y_test))
r2.append(results)

0.9999996595348577 0.771843970356952


In [72]:
gsc10_1 = GridSearchCV(
            estimator=xgb.XGBRegressor(),
            param_grid={"n_estimators": [300],
                        "learning_rate": [0.15, 0.2],
                        "max_depth": [20],
                        "min_child_weight": [1],
                        "gamma":[ 0.0],
                        "colsample_bytree":[0.4],},
            cv=5, scoring='neg_root_mean_squared_error', verbose=0, n_jobs=-1)

grid_result10_1 = gsc10_1.fit(X_train, y_train)

In [73]:
print( grid_result10_1.score(X_train,y_train), grid_result10_1.score(X_test, y_test))
results = (grid_result10_1.score(X_train,y_train), grid_result10_1.score(X_test, y_test))
rmse.append(results)

-12.178884170999162 -6493.3579260255465


In [74]:
#grid_result10.best_params_

**RESULTATS**

In [75]:
cols = ['train test aleatori', 'train test 14-20/21', 'train test aleatori + no campanya', 'train test aleatori + no superficie', 'train test 14-20/21 + no campanya', 'train test 14-20/21 + no estació', 'train test 14-20/21 + no superficie', 'train test 14-19+21/20 + no superficie', 'train test 14-19+21/20 + no campaña', 'train test 19-20/21 no zona']

results = pd.DataFrame([r2,rmse], columns = cols).transpose()
results['r2'] = results[0]
results['rmse'] = results[1]
results.drop(columns=[0,1], inplace=True)


In [76]:
results

,r2,rmse
train test aleatori,"(0.9999992561341488, 0.8178248625881704)","(-11.42461684042509, -5715.316533858098)"
train test 14-20/21,"(0.9999985998053591, 0.785189512327019)","(-15.633273106167826, -6322.651292719792)"
train test aleatori + no campanya,"(0.9903187624290571, 0.8253618181314967)","(-1303.3461439260245, -5595.840612606942)"
train test aleatori + no superficie,"(0.9743104800597309, 0.7657780012920226)","(-2123.109174125441, -6480.515841173995)"
train test 14-20/21 + no campanya,"(0.9992726156128926, 0.7769533794143471)","(-356.31794649555417, -6442.720832617403)"
train test 14-20/21 + no estació,"(0.999999699878823, 0.7717865752773425)","(-7.237757529993546, -6516.915410157708)"
train test 14-20/21 + no superficie,"(0.9721059456678479, 0.7795763774538554)","(-2206.5386072009433, -6404.726036792351)"
train test 14-19+21/20 + no superficie,"(0.9726474615267175, 0.7752420512855773)","(-2190.7390166606096, -6364.039637393608)"
train test 14-19+21/20 + no campaña,"(0.9992600655510594, 0.7535613992274898)","(-360.32006551523733, -6663.9190550972535)"
train test 19-20/21 no zona,"(0.9999996595348577, 0.771843970356952)","(-12.178884170999162, -6493.3579260255465)"




*   Agafar dataset bo ✅
*   Agafar dadaes meteorològiques



**XGBOOST TRAIN + DADES METEO**



+ Assignar a cada combinació diferent d'any i estació la suma de la producció d'aquell any i estació en el train. Encara q no crec q estigui del tot bé.

In [9]:
df_meteo = pd.read_csv("/content/df_meteo_eto.csv", index_col = False, parse_dates = ['validTimeUtc'], dayfirst=True)
df_meteo

,validTimeUtc,ID_ESTACION,precip1HourDayAvg,pressureChangeDayAvg,relativeHumidityDayAvg,snow1HourDayAvg,temperatureDayAvg,temperatureDewPointDayAvg,temperatureFeelsLikeDayAvg,uvIndexDayAvg,...,WindSpeedLocalEveningMin,WindSpeedLocalMorningAvg,WindSpeedLocalMorningMax,WindSpeedLocalMorningMin,WindSpeedLocalNighttimeAvg,WindSpeedLocalNighttimeMax,WindSpeedLocalNighttimeMin,WindSpeedLocalOvernightAvg,WindSpeedLocalOvernightMax,WindSpeedLocalOvernightMin
0,2015-06-30,0,0.0,-0.321739,44.700000,0.0,26.534783,12.095652,26.565217,2.521739,...,1.8,2.1,3.5,1.2,2.4,4.7,1.3,1.5,1.9,1.3
1,2015-06-30,1,0.0,-0.308696,38.773913,0.0,26.834783,10.226087,26.834783,2.652174,...,2.3,2.4,4.0,1.4,2.9,5.4,1.6,1.7,2.0,1.6
2,2015-06-30,2,0.0,-0.330435,44.865217,0.0,26.821739,12.252174,26.839130,2.565217,...,2.0,2.1,3.6,1.3,2.5,4.8,1.3,1.5,1.8,1.3
3,2015-06-30,3,0.0,-0.321739,41.652174,0.0,27.213043,11.456522,27.213043,2.565217,...,2.1,2.2,3.8,1.4,2.7,5.2,1.4,1.6,1.9,1.4
4,2015-06-30,4,0.0,-0.308696,40.960870,0.0,26.852174,10.904348,26.865217,2.565217,...,2.3,2.3,4.0,1.5,2.9,5.4,1.5,1.6,1.9,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51155,2022-06-30,15,0.0,0.230435,53.113043,0.0,23.769565,12.200000,23.791304,2.565217,...,2.2,1.2,1.6,0.7,3.3,4.1,2.2,3.8,4.1,3.2
51156,2022-06-30,16,0.0,0.239130,52.556522,0.0,24.182609,12.626087,24.200000,2.565217,...,1.7,1.0,1.2,0.5,3.1,4.0,1.7,3.7,4.0,3.2
51157,2022-06-30,17,0.0,0.226087,48.439130,0.0,23.582609,10.091304,23.586957,2.652174,...,2.3,1.3,1.6,1.0,3.3,4.5,2.3,3.3,3.7,2.6
51158,2022-06-30,18,0.0,0.304348,52.656522,0.0,24.195652,13.030435,24.239130,2.521739,...,1.7,1.3,1.7,0.9,2.6,3.3,1.7,3.1,3.3,2.6


In [10]:
df_meteo['DIA'] = df_meteo.validTimeUtc.dt.day
df_meteo['MES'] = df_meteo.validTimeUtc.dt.month
df_meteo['CAMPAÑA'] = df_meteo.validTimeUtc.dt.year-2000

In [11]:
df_meteo.drop(columns=['validTimeUtc'], inplace= True)

In [12]:
df_meteo.head()

,ID_ESTACION,precip1HourDayAvg,pressureChangeDayAvg,relativeHumidityDayAvg,snow1HourDayAvg,temperatureDayAvg,temperatureDewPointDayAvg,temperatureFeelsLikeDayAvg,uvIndexDayAvg,visibilityDayAvg,...,WindSpeedLocalMorningMin,WindSpeedLocalNighttimeAvg,WindSpeedLocalNighttimeMax,WindSpeedLocalNighttimeMin,WindSpeedLocalOvernightAvg,WindSpeedLocalOvernightMax,WindSpeedLocalOvernightMin,DIA,MES,CAMPAÑA
0,0,0.0,-0.321739,44.700000,0.0,26.534783,12.095652,26.565217,2.521739,16.09,...,1.2,2.4,4.7,1.3,1.5,1.9,1.3,30,6,15
1,1,0.0,-0.308696,38.773913,0.0,26.834783,10.226087,26.834783,2.652174,16.09,...,1.4,2.9,5.4,1.6,1.7,2.0,1.6,30,6,15
2,2,0.0,-0.330435,44.865217,0.0,26.821739,12.252174,26.839130,2.565217,16.09,...,1.3,2.5,4.8,1.3,1.5,1.8,1.3,30,6,15
3,3,0.0,-0.321739,41.652174,0.0,27.213043,11.456522,27.213043,2.565217,16.09,...,1.4,2.7,5.2,1.4,1.6,1.9,1.4,30,6,15
4,4,0.0,-0.308696,40.960870,0.0,26.852174,10.904348,26.865217,2.565217,16.09,...,1.5,2.9,5.4,1.5,1.6,1.9,1.5,30,6,15


In [47]:
#df_merge = df_meteo.copy()
#df_merge['PRODUCCIO'] = df[df.loc[:,['ID_ESTACION','CAMPAÑA']] == df_meteo.loc[:,['ID_ESTACION', 'CAMPAÑA']]]['PRODUCCION'].sum()

ValueError: ignored

In [19]:
sum_prod = df.groupby(by=['ID_ESTACION', 'CAMPAÑA']).sum()['PRODUCCION']
sum_prod = pd.DataFrame(sum_prod).reset_index()

In [29]:
sum_prod.groupby(by='CAMPAÑA').count()

,ID_ESTACION,PRODUCCION
CAMPAÑA,,
14,18,18
15,16,16
16,16,16
17,18,18
18,17,17
19,16,16
20,17,17
21,16,16


In [ ]:
df_meteo.groupby(by=['ID_ESTACION', 'CAMPAÑA']).sum()

In [24]:
for est, camp, prod in sum_prod.values:
  

0.0 14.0
0.0 20.0
1.0 17.0
1.0 18.0
2.0 14.0
2.0 15.0
2.0 16.0
2.0 17.0
2.0 18.0
2.0 19.0
2.0 20.0
2.0 21.0
3.0 14.0
3.0 15.0
3.0 16.0
3.0 17.0
3.0 18.0
3.0 19.0
3.0 20.0
3.0 21.0
4.0 14.0
5.0 14.0
5.0 15.0
5.0 16.0
5.0 17.0
5.0 18.0
5.0 19.0
5.0 20.0
5.0 21.0
6.0 14.0
6.0 15.0
6.0 16.0
6.0 17.0
6.0 18.0
6.0 19.0
6.0 20.0
6.0 21.0
7.0 14.0
7.0 15.0
7.0 16.0
7.0 17.0
7.0 18.0
7.0 19.0
7.0 20.0
7.0 21.0
8.0 14.0
8.0 15.0
8.0 16.0
8.0 17.0
8.0 18.0
8.0 19.0
8.0 20.0
8.0 21.0
9.0 14.0
9.0 15.0
9.0 16.0
9.0 17.0
9.0 18.0
9.0 19.0
9.0 20.0
9.0 21.0
10.0 14.0
10.0 15.0
10.0 16.0
10.0 17.0
10.0 18.0
10.0 19.0
10.0 20.0
10.0 21.0
11.0 14.0
11.0 17.0
11.0 18.0
11.0 19.0
11.0 20.0
11.0 21.0
12.0 14.0
12.0 15.0
12.0 16.0
12.0 17.0
12.0 18.0
12.0 19.0
12.0 20.0
12.0 21.0
13.0 15.0
13.0 16.0
13.0 17.0
13.0 18.0
13.0 19.0
13.0 20.0
13.0 21.0
14.0 14.0
14.0 15.0
14.0 16.0
14.0 17.0
14.0 18.0
14.0 19.0
14.0 20.0
14.0 21.0
15.0 14.0
15.0 15.0
15.0 16.0
15.0 17.0
15.0 18.0
15.0 19.0
15.0 20.0
15.0 21.0
1

**XGBOOST TRAIN + XGBOOST DADES METEO**

**XGBOOST (TRAIN + DADES METEO) PCA**

In [93]:
#from sklearn.decomposition import PCA

#pca = PCA()
#pca = pca.fit(df_meteo)
#pca_meteo = pca.transform(df_meteo)
#pca_meteo.shape


(51160, 403)

**CATEGORITZAR SUPERFICIE:**

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv("/DATASETS/D_T_3.csv")
df.drop(columns=[df.columns[0]], inplace=True)
X = df.drop(columns=['SUPERFICIE'])
y = df['SUPERFICIE']

FileNotFoundError: ignored

In [ ]:
l_enc = LabelEncoder()
#y = l_enc.fit_transform(y)
#np.unique(y)

In [53]:
df_train = df.copy()
df_train['SUPERFICIE_CAT'] = l_enc.fit_transform(df_train['SUPERFICIE'])
df_train.drop(columns=['SUPERFICIE'], inplace=True)
df_supcat = df_train.groupby(by='SUPERFICIE_CAT').count().reset_index()
sup_uniq = df_supcat[df_supcat['CAMPAÑA']==1]['SUPERFICIE_CAT']
df_train2 = df_train[df_train['SUPERFICIE_CAT'].isin(sup_uniq)]
df_train2

,Unnamed: 0,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,PRODUCCION,ALTITUD_MIN,ALTITUD_DIF,SUPERFICIE_CAT
2,2,14,85579,340,4,520,32,1,0,1,20978.0,520.000000,0.00000,4528
3,3,14,69671,340,4,520,32,1,0,1,40722.0,520.000000,0.00000,5234
4,4,14,14001,852,14,650-660,81,0,0,1,14126.0,654.501718,9.19244,5353
5,5,14,17059,852,14,650-660,81,0,0,1,6054.0,654.501718,9.19244,4667
6,6,14,87611,602,14,650-660,81,0,0,1,12900.0,654.501718,9.19244,5248
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8501,8501,21,69003,349,19,445-465,59,0,0,1,26710.0,445.000000,20.00000,6897
8512,8512,21,49636,473,10,450-480,9,1,0,0,4530.0,450.000000,30.00000,5766
8514,8514,21,49636,473,10,450-480,26,1,0,1,8020.0,450.000000,30.00000,5777
8515,8515,21,49636,473,10,450-480,52,1,0,1,36630.0,450.000000,30.00000,7082


In [ ]:
df_train.drop(columns=['SUPERFICIE'], inplace=True)
df_supcat = df_train.groupby(by='SUPERFICIE_CAT').count().reset_index()
index = df_supcat[df_supcat['CAMPAÑA']==1]['SUPERFICIE_CAT']

df_supcat = df_supcat[df_supcat['CAMPAÑA']==1]
df_supcat

df_supcat = df_supcat[df_supcat['CAMPAÑA']==1].reset_index().reset_index()
base_train = list(df_supcat['index'])
X_train = df_train.iloc[base_train]
df_train.drop(index = base_train, inplace = True)
df_train.groupby(by='SUPERFICIE_CAT').count()
df_supcat

In [44]:
X_train

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,PRODUCCION,ALTITUD_MIN,ALTITUD_DIF,SUPERFICIE_CAT
0,14,76953,515,4,660,26,1,0,1,22215.000,660.000000,0.00000,7225
1,14,84318,515,4,660,26,1,0,1,22215.000,660.000000,0.00000,7225
2,14,85579,340,4,520,32,1,0,1,20978.000,520.000000,0.00000,7037
3,14,69671,340,4,520,32,1,0,1,40722.000,520.000000,0.00000,7151
4,14,14001,852,14,650-660,81,0,0,1,14126.000,654.501718,9.19244,7038
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6584,20,38272,44,7,620-640,71,0,0,1,498.528,620.000000,20.00000,2748
6585,20,38272,44,7,620-640,87,0,0,0,2960.000,620.000000,20.00000,4056
6586,20,38272,44,7,620-640,17,0,0,1,7580.000,620.000000,20.00000,3660
6587,20,38272,44,7,620-640,81,0,0,1,9200.000,620.000000,20.00000,5698


In [80]:
df_train.groupby(by='SUPERFICIE_CAT').count()

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,PRODUCCION,ALTITUD_MIN,ALTITUD_DIF
SUPERFICIE_CAT,,,,,,,,,,,,
12,1,1,1,1,1,1,1,1,1,1,1,1
17,1,1,1,1,1,1,1,1,1,1,1,1
18,1,1,1,1,1,1,1,1,1,1,1,1
20,1,1,1,1,1,1,1,1,1,1,1,1
24,1,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
7419,2,2,2,2,2,2,2,2,2,2,2,2
7422,1,1,1,1,1,1,1,1,1,1,1,1
7425,1,1,1,1,1,1,1,1,1,1,1,1


In [82]:
df.shape, df_train.shape

((8526, 13), (1937, 13))

In [79]:
df_train2, df_test= train_test_split(df_train, 
                                    test_size= 0.15,
                                    shuffle= True, 
                                    random_state= 0, 
                                    stratify=df_train[["SUPERFICIE_CAT"]])

ValueError: ignored

+ xgboost x predir superficie categòrica

In [62]:
gsc_sup = GridSearchCV(
            estimator=xgb.XGBClassifier(),
            param_grid={"n_estimators": [100, 300, 500],
                        "learning_rate": [0.15],
                        "max_depth": [3, 20, 40, 60],
                        "min_child_weight": [1],
                        "gamma":[ 0.0],
                        "colsample_bytree":[0.4,1],},
            cv=5, scoring='neg_root_mean_squared_error', verbose=0, n_jobs=-1)

grid_result_sup = gsc_sup.fit(X_train, y_train)
#xgb_model = xgb.XGBClassifier(objective="multi:softprob", random_state=42)
#xgb_model.fit(X, y)

#y_pred = xgb_model.predict(X)

#print(confusion_matrix(y, y_pred))
#(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
#       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
#       n_estimators=100, n_jobs=1, nthread=None,
#       objective='multi:softprob', random_state=0, reg_alpha=0,
#       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
#       subsample=1, verbosity=1)

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


ValueError: ignored

+ logistic x predir superficie categòrica


+ o rf?